## 1. 📦 Instalación de Dependencias

Instalamos todas las librerías necesarias para el análisis estadístico avanzado.

In [ ]:
# Instalación de librerías para análisis estadístico avanzado
import subprocess
import sys

def install_package(package):
    """Instala un paquete usando pip si no está disponible"""
    try:
        __import__(package.split('==')[0])
        print(f"✅ {package} ya está instalado")
    except ImportError:
        print(f"📦 Instalando {package}...")
        subprocess.check_call([sys.executable, "-m", "pip", "install", package])
        print(f"✅ {package} instalado correctamente")

# Lista de paquetes necesarios
packages = [
    "pandas>=1.5.0",
    "numpy>=1.21.0", 
    "matplotlib>=3.5.0",
    "seaborn>=0.11.0",
    "plotly>=5.0.0",
    "scipy>=1.9.0",
    "scikit-learn>=1.1.0",
    "statsmodels>=0.13.0",
    "pingouin>=0.5.0",  # Para análisis estadísticos avanzados
    "kaleido",  # Para exportar gráficos de plotly
    "jupyter-dash",  # Para dashboards interactivos
    "umap-learn",  # Para reducción de dimensionalidad
    "yellowbrick",  # Para visualizaciones de ML
    "missingno"  # Para análisis de datos faltantes
]

print("🚀 Iniciando instalación de dependencias...")
for package in packages:
    install_package(package)

print("\n🎉 Todas las dependencias han sido instaladas correctamente!")
print("📋 Reinicia el kernel si es necesario para cargar las nuevas librerías.")

## 2. 📚 Importación de Librerías

Importamos todas las librerías necesarias para el análisis estadístico, visualización y machine learning.

In [ ]:
# Librerías fundamentales
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

# Análisis estadístico
import scipy.stats as stats
from scipy.stats import shapiro, kstest, mannwhitneyu, wilcoxon, ttest_rel, ttest_ind
from scipy.stats import pearsonr, spearmanr, chi2_contingency
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.multivariate.manova import MANOVA
import pingouin as pg

# Machine Learning
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score
from sklearn.feature_selection import SelectKBest, f_classif
import umap

# Visualización de ML
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer
from yellowbrick.features import RadViz, ParallelCoordinates

# Utilidades
import os
from datetime import datetime
import json
import missingno as msno
from itertools import combinations
from functools import partial

# Configuraciones de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

# Configuraciones de plotly
import plotly.io as pio
pio.templates.default = "plotly_white"

# Colores personalizados para visualizaciones
COLORS = {
    'AP1': '#FF6B6B',    # Rojo suave
    'AP2': '#4ECDC4',    # Verde azulado
    'male': '#3498DB',   # Azul
    'female': '#E74C3C', # Rojo
    'primary': '#2C3E50',
    'secondary': '#95A5A6',
    'success': '#27AE60',
    'warning': '#F39C12',
    'danger': '#E74C3C'
}

print("📚 Librerías importadas correctamente")
print(f"🐼 Pandas: {pd.__version__}")
print(f"🔢 NumPy: {np.__version__}")
print(f"📊 Matplotlib: {plt.matplotlib.__version__}")
print(f"🎨 Seaborn: {sns.__version__}")
print("✅ Configuraciones aplicadas")

# Configurar semilla para reproducibilidad
np.random.seed(42)
import random
random.seed(42)

## 3. 📊 Carga y Preparación de Datos

Cargamos los datasets de métricas de SonarCloud y datos de issues para el análisis.

In [ ]:
# Función para cargar datos con manejo de errores
def load_data_safely(file_path, description="archivo"):
    """Carga un archivo CSV con manejo seguro de errores"""
    try:
        if os.path.exists(file_path):
            df = pd.read_csv(file_path, encoding='utf-8-sig')
            print(f"✅ {description} cargado: {file_path}")
            print(f"   📊 Dimensiones: {df.shape}")
            return df
        else:
            print(f"❌ Archivo no encontrado: {file_path}")
            return pd.DataFrame()
    except Exception as e:
        print(f"❌ Error cargando {description}: {str(e)}")
        return pd.DataFrame()

def download_data_from_github(url, description="archivo"):
    """Descarga datos directamente desde GitHub con manejo de errores"""
    try:
        print(f"🌐 Descargando {description} desde GitHub...")
        df = pd.read_csv(url, encoding='utf-8-sig')
        print(f"✅ {description} descargado exitosamente")
        print(f"   📊 Dimensiones: {df.shape}")
        return df
    except Exception as e:
        print(f"❌ Error descargando {description}: {str(e)}")
        return pd.DataFrame()

# Cargar dataset principal de métricas desde GitHub
print("🔄 Cargando datasets...")
github_url = "https://raw.githubusercontent.com/TesisEnel/Recopilacion_Datos_CalidadCodigo/refs/heads/main/Estudiantes_2023-2024_con_metricas_sonarcloud.csv"

# Intentar descargar desde GitHub primero
df_metricas = download_data_from_github(
    github_url,
    'Dataset de métricas de SonarCloud'
)

# Si falla la descarga, intentar cargar desde archivo local como respaldo
if df_metricas.empty:
    print("⚠️ Descarga desde GitHub falló. Intentando cargar archivo local...")
    df_metricas = load_data_safely(
        '../data/Estudiantes_2023-2024_con_metricas_sonarcloud.csv',
        'Dataset local de métricas de SonarCloud'
    )

# Buscar y cargar el archivo de issues más reciente
data_dir = '../data'
issues_files = []
if os.path.exists(data_dir):
    for file in os.listdir(data_dir):
        if file.startswith('issues_detallados') and file.endswith('.csv'):
            issues_files.append(file)

# Cargar archivo de issues más reciente o el latest
if issues_files:
    # Buscar primero el archivo 'latest'
    latest_file = [f for f in issues_files if 'latest' in f]
    if latest_file:
        issues_file = latest_file[0]
    else:
        # Si no hay latest, tomar el más reciente por fecha
        issues_files.sort(reverse=True)
        issues_file = issues_files[0]
    
    df_issues = load_data_safely(
        f'{data_dir}/{issues_file}',
        'Dataset de issues detallados'
    )
else:
    print("⚠️ No se encontraron archivos de issues. Se procederá solo con métricas.")
    df_issues = pd.DataFrame()

# Verificar carga exitosa
if not df_metricas.empty:
    print(f"\n📋 Dataset de métricas:")
    print(f"   👥 Estudiantes: {df_metricas['Id'].nunique()}")
    print(f"   📊 Columnas: {len(df_metricas.columns)}")
    print(f"   📅 Semestres: {df_metricas['Semestre'].unique().tolist()}")
    
    # Mostrar primeras columnas de métricas
    print(f"\n🔍 Primeras 5 filas de métricas:")
    display(df_metricas[['Id', 'Estudiante', 'Sexo', 'Semestre', 'bugs_AP1', 'bugs_AP2', 
                        'code_smells_AP1', 'code_smells_AP2', 'ncloc_AP1', 'ncloc_AP2']].head())

if not df_issues.empty:
    print(f"\n📋 Dataset de issues:")
    print(f"   🐛 Total issues: {len(df_issues)}")
    print(f"   👥 Estudiantes: {df_issues['student_id'].nunique()}")
    print(f"   📁 Proyectos: {df_issues['project_key'].nunique()}")
    
    # Distribución por asignación
    if 'assignment' in df_issues.columns:
        print(f"   📚 Por asignación:")
        for assignment in df_issues['assignment'].value_counts().items():
            print(f"      {assignment[0]}: {assignment[1]} issues")
    
    print(f"\n🔍 Muestra de issues:")
    display(df_issues[['student_id', 'assignment', 'type', 'severity', 'rule']].head())

else:
    print("⚠️ No se cargaron datos de issues - se continuará con métricas únicamente")

print(f"\n✅ Carga de datos completada")
print(f"📊 Total de datasets cargados: {1 if not df_metricas.empty else 0} + {1 if not df_issues.empty else 0}")
print(f"🌐 Fuente de datos: {'GitHub (actualizada)' if not df_metricas.empty else 'Local/Error'}")

### 3.1 Preprocesamiento y Limpieza de Datos

Preparamos los datos para el análisis estadístico, incluyendo limpieza, transformaciones y creación de variables derivadas.

In [ ]:
if not df_metricas.empty:
    print("🔄 Iniciando preprocesamiento de datos...")
    
    # Crear una copia para trabajar
    df = df_metricas.copy()
    
    # 1. LIMPIEZA BÁSICA
    print("\n1️⃣ Limpieza básica de datos")
    
    # Renombrar columnas para mayor claridad
    df['estudiante_id'] = df['Id']
    df['nombre'] = df['Estudiante']
    df['genero'] = df['Sexo'].map({1: 'Masculino', 2: 'Femenino'})
    df['semestre'] = df['Semestre']
    
    print(f"   ✅ Renombrado de columnas completado")
    print(f"   👥 Distribución por género: {df['genero'].value_counts().to_dict()}")
    print(f"   📅 Distribución por semestre: {df['semestre'].value_counts().to_dict()}")
    
    # 2. IDENTIFICAR MÉTRICAS DE CALIDAD
    print("\n2️⃣ Identificación de métricas de calidad")
    
    # Métricas principales de SonarCloud
    metricas_calidad = {
        'bugs': ['bugs_AP1', 'bugs_AP2'],
        'vulnerabilities': ['vulnerabilities_AP1', 'vulnerabilities_AP2'],
        'security_hotspots': ['security_hotspots_AP1', 'security_hotspots_AP2'],
        'code_smells': ['code_smells_AP1', 'code_smells_AP2'],
        'technical_debt': ['technical_debt_AP1', 'technical_debt_AP2'],
        'complexity': ['complexity_AP1', 'complexity_AP2'],
        'cognitive_complexity': ['cognitive_complexity_AP1', 'cognitive_complexity_AP2'],
        'ncloc': ['ncloc_AP1', 'ncloc_AP2'],
        'duplicated_lines_density': ['duplicated_lines_density_AP1', 'duplicated_lines_density_AP2'],
        'coverage': ['coverage_AP1', 'coverage_AP2'],
        'comment_lines_density': ['comment_lines_density_AP1', 'comment_lines_density_AP2'],
        'open_issues': ['open_issues_AP1', 'open_issues_AP2']
    }
    
    # Verificar qué métricas están disponibles
    metricas_disponibles = {}
    for metrica, columnas in metricas_calidad.items():
        cols_existentes = [col for col in columnas if col in df.columns]
        if cols_existentes:
            metricas_disponibles[metrica] = cols_existentes
            print(f"   ✅ {metrica}: {len(cols_existentes)} columnas disponibles")
        else:
            print(f"   ❌ {metrica}: No disponible")
    
    # 3. MANEJO DE DATOS FALTANTES
    print(f"\n3️⃣ Análisis de datos faltantes")
    
    # Analizar patrones de datos faltantes
    missing_data = df.isnull().sum()
    missing_metrics = missing_data[missing_data > 0]
    
    if len(missing_metrics) > 0:
        print(f"   📊 Columnas con datos faltantes:")
        for col, count in missing_metrics.items():
            percentage = (count / len(df)) * 100
            print(f"      {col}: {count} ({percentage:.1f}%)")
    else:
        print(f"   ✅ No se encontraron datos faltantes")
    
    # 4. CREAR VARIABLES DERIVADAS
    print(f"\n4️⃣ Creación de variables derivadas")
    
    # Calcular métricas de mejora (diferencia AP2 - AP1)
    for metrica, columnas in metricas_disponibles.items():
        if len(columnas) == 2:  # Si tenemos ambas asignaciones
            ap1_col, ap2_col = columnas
            
            # Mejora absoluta
            df[f'{metrica}_mejora_abs'] = df[ap2_col] - df[ap1_col]
            
            # Mejora relativa (solo para métricas donde más bajo es mejor)
            if metrica in ['bugs', 'vulnerabilities', 'security_hotspots', 'code_smells']:
                # Para estas métricas, reducción es mejora
                df[f'{metrica}_mejora_rel'] = ((df[ap1_col] - df[ap2_col]) / (df[ap1_col] + 1)) * 100
            elif metrica in ['coverage', 'comment_lines_density']:
                # Para estas métricas, aumento es mejora
                df[f'{metrica}_mejora_rel'] = ((df[ap2_col] - df[ap1_col]) / (df[ap1_col] + 1)) * 100
            
            print(f"   ✅ Variables de mejora creadas para {metrica}")
    
    # 5. NORMALIZACIÓN POR TAMAÑO DE CÓDIGO
    print(f"\n5️⃣ Normalización por tamaño de código")
    
    # Crear métricas normalizadas por NCLOC (líneas de código)
    if 'ncloc' in metricas_disponibles:
        for metrica in ['bugs', 'vulnerabilities', 'code_smells']:
            if metrica in metricas_disponibles:
                for asignacion in ['AP1', 'AP2']:
                    metrica_col = f'{metrica}_{asignacion}'
                    ncloc_col = f'ncloc_{asignacion}'
                    
                    if metrica_col in df.columns and ncloc_col in df.columns:
                        # Densidad de defectos por 1000 líneas de código
                        df[f'{metrica}_density_{asignacion}'] = (df[metrica_col] / (df[ncloc_col] + 1)) * 1000
                        
                print(f"   ✅ Densidad calculada para {metrica}")
    
    # 6. CREAR ÍNDICES COMPUESTOS
    print(f"\n6️⃣ Creación de índices compuestos de calidad")
    
    # Índice de calidad general (menor es mejor)
    for asignacion in ['AP1', 'AP2']:
        quality_components = []
        
        # Componentes del índice (normalizados)
        if f'bugs_{asignacion}' in df.columns and f'ncloc_{asignacion}' in df.columns:
            df[f'bugs_norm_{asignacion}'] = df[f'bugs_{asignacion}'] / (df[f'ncloc_{asignacion}'] / 1000 + 1)
            quality_components.append(f'bugs_norm_{asignacion}')
        
        if f'vulnerabilities_{asignacion}' in df.columns and f'ncloc_{asignacion}' in df.columns:
            df[f'vuln_norm_{asignacion}'] = df[f'vulnerabilities_{asignacion}'] / (df[f'ncloc_{asignacion}'] / 1000 + 1)
            quality_components.append(f'vuln_norm_{asignacion}')
        
        if f'code_smells_{asignacion}' in df.columns and f'ncloc_{asignacion}' in df.columns:
            df[f'smells_norm_{asignacion}'] = df[f'code_smells_{asignacion}'] / (df[f'ncloc_{asignacion}'] / 1000 + 1)
            quality_components.append(f'smells_norm_{asignacion}')
        
        # Calcular índice compuesto
        if quality_components:
            df[f'quality_index_{asignacion}'] = df[quality_components].sum(axis=1)
            print(f"   ✅ Índice de calidad calculado para {asignacion}")
    
    # 7. IDENTIFICAR ESTUDIANTES CON AMBOS PROYECTOS
    print(f"\n7️⃣ Identificación de estudiantes con datos pareados")
    
    # Filtrar estudiantes que tienen datos en ambas asignaciones
    estudiantes_completos = []
    for metrica in ['bugs', 'code_smells', 'ncloc']:
        if metrica in metricas_disponibles and len(metricas_disponibles[metrica]) == 2:
            ap1_col, ap2_col = metricas_disponibles[metrica]
            mask_completo = df[ap1_col].notna() & df[ap2_col].notna()
            estudiantes_con_metrica = df[mask_completo]['estudiante_id'].tolist()
            estudiantes_completos.extend(estudiantes_con_metrica)
    
    # Estudiantes que aparecen en todas las métricas principales
    estudiantes_pareados = list(set(estudiantes_completos))
    df['datos_pareados'] = df['estudiante_id'].isin(estudiantes_pareados)
    
    print(f"   👥 Total estudiantes: {len(df)}")
    print(f"   👥 Estudiantes con datos pareados: {df['datos_pareados'].sum()}")
    print(f"   👥 Estudiantes solo con AP1 o AP2: {(~df['datos_pareados']).sum()}")
    
    # 8. RESUMEN FINAL
    print(f"\n✅ Preprocesamiento completado")
    print(f"📊 Dataset final: {df.shape}")
    print(f"📋 Métricas de calidad disponibles: {len(metricas_disponibles)}")
    print(f"📈 Variables derivadas creadas: {len([col for col in df.columns if 'mejora' in col or 'density' in col or 'quality_index' in col])}")
    
    # Mostrar estadísticas básicas
    print(f"\n📊 Estadísticas básicas de métricas principales:")
    main_metrics = ['bugs_AP1', 'bugs_AP2', 'code_smells_AP1', 'code_smells_AP2', 'ncloc_AP1', 'ncloc_AP2']
    available_metrics = [col for col in main_metrics if col in df.columns]
    if available_metrics:
        display(df[available_metrics].describe())
    
else:
    print("❌ No se puede proceder sin datos de métricas")
    df = pd.DataFrame()

## 4. 🔍 Análisis Exploratorio de Datos (EDA)

En esta sección realizamos un análisis exploratorio comprehensivo de las métricas de calidad de código, incluyendo distribuciones, patrones y relaciones entre variables.

In [ ]:
# Funciones para análisis exploratorio
def create_distribution_plots(data, metrics, title_prefix="Distribución"):
    """Crea gráficos de distribución para múltiples métricas"""
    n_metrics = len(metrics)
    n_cols = 3
    n_rows = (n_metrics + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 5 * n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes
    
    for i, metric in enumerate(metrics):
        if metric in data.columns:
            ax = axes[i]
            # Histograma con curva de densidad
            data[metric].hist(bins=30, alpha=0.7, ax=ax, color=COLORS['primary'])
            data[metric].plot.density(ax=ax, color=COLORS['danger'], linewidth=2)
            ax.set_title(f'{title_prefix}: {metric}')
            ax.set_xlabel('Valor')
            ax.set_ylabel('Frecuencia / Densidad')
            ax.grid(True, alpha=0.3)
            
            # Estadísticas en el gráfico
            mean_val = data[metric].mean()
            median_val = data[metric].median()
            ax.axvline(mean_val, color='red', linestyle='--', label=f'Media: {mean_val:.2f}')
            ax.axvline(median_val, color='green', linestyle='--', label=f'Mediana: {median_val:.2f}')
            ax.legend()
        else:
            axes[i].text(0.5, 0.5, f'Métrica {metric}\\nno disponible', 
                        ha='center', va='center', transform=axes[i].transAxes)
            axes[i].set_title(f'{metric} - No disponible')
    
    # Ocultar ejes sobrantes
    for i in range(len(metrics), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()

def create_comparison_boxplots(data, metrics, assignment_cols=['AP1', 'AP2']):
    """Crea boxplots comparativos entre asignaciones"""
    n_metrics = len(metrics)
    n_cols = 2
    n_rows = (n_metrics + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(15, 6 * n_rows))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_cols == 1 else axes
    
    for i, metric in enumerate(metrics):
        ax = axes[i]
        
        # Preparar datos para boxplot
        metric_data = []
        labels = []
        
        for assignment in assignment_cols:
            col_name = f'{metric}_{assignment}'
            if col_name in data.columns:
                values = data[col_name].dropna()
                metric_data.append(values)
                labels.append(f'{metric}\\n{assignment}')
        
        if metric_data:
            bp = ax.boxplot(metric_data, labels=labels, patch_artist=True)
            
            # Colorear boxplots
            colors = [COLORS['AP1'], COLORS['AP2']]
            for patch, color in zip(bp['boxes'], colors[:len(bp['boxes'])]):
                patch.set_facecolor(color)
                patch.set_alpha(0.7)
            
            ax.set_title(f'Comparación {metric}: AP1 vs AP2')
            ax.grid(True, alpha=0.3)
            
            # Agregar estadísticas
            for j, values in enumerate(metric_data):
                median_val = values.median()
                mean_val = values.mean()
                ax.text(j+1, median_val, f'Med: {median_val:.1f}', 
                       ha='center', va='bottom', fontsize=9)
        else:
            ax.text(0.5, 0.5, f'Datos no\\ndisponibles', 
                   ha='center', va='center', transform=ax.transAxes)
            ax.set_title(f'{metric} - Sin datos')
    
    # Ocultar ejes sobrantes
    for i in range(len(metrics), len(axes)):
        axes[i].set_visible(False)
    
    plt.tight_layout()
    plt.show()

# Ejecutar análisis exploratorio si tenemos datos
if not df.empty:
    print("🔍 Iniciando Análisis Exploratorio de Datos...")
    
    # 1. ESTADÍSTICAS DESCRIPTIVAS GENERALES
    print("\\n1️⃣ Estadísticas Descriptivas Generales")
    print("="*60)
    
    print(f"👥 Total de estudiantes: {len(df)}")
    print(f"📊 Distribución por género:")
    print(df['genero'].value_counts())
    print(f"\\n📅 Distribución por semestre:")
    print(df['semestre'].value_counts())
    
    # 2. ANÁLISIS DE MÉTRICAS PRINCIPALES
    print("\\n2️⃣ Análisis de Métricas Principales de Calidad")
    print("="*60)
    
    # Métricas clave para analizar
    main_quality_metrics = ['bugs', 'code_smells', 'vulnerabilities', 'ncloc', 'complexity']
    
    # Estadísticas por asignación
    for assignment in ['AP1', 'AP2']:
        print(f"\\n📚 Estadísticas para {assignment}:")
        assignment_metrics = [f'{metric}_{assignment}' for metric in main_quality_metrics]
        available_assignment_metrics = [col for col in assignment_metrics if col in df.columns]
        
        if available_assignment_metrics:
            stats_df = df[available_assignment_metrics].describe()
            display(stats_df)
            
            # Identificar outliers
            print(f"\\n🎯 Outliers en {assignment} (valores > Q3 + 1.5*IQR):")
            for col in available_assignment_metrics:
                q1 = df[col].quantile(0.25)
                q3 = df[col].quantile(0.75)
                iqr = q3 - q1
                outlier_threshold = q3 + 1.5 * iqr
                outliers = df[df[col] > outlier_threshold][col]
                if len(outliers) > 0:
                    print(f"   {col}: {len(outliers)} outliers (max: {outliers.max():.1f})")
    
    # 3. DISTRIBUCIONES DE MÉTRICAS CLAVE
    print("\\n3️⃣ Visualización de Distribuciones")
    print("="*60)
    
    # Gráficos de distribución para AP1
    ap1_metrics = [f'{metric}_AP1' for metric in main_quality_metrics if f'{metric}_AP1' in df.columns]
    if ap1_metrics:
        print("📊 Distribuciones en AP1:")
        create_distribution_plots(df, ap1_metrics, "Distribución AP1")
    
    # Gráficos de distribución para AP2  
    ap2_metrics = [f'{metric}_AP2' for metric in main_quality_metrics if f'{metric}_AP2' in df.columns]
    if ap2_metrics:
        print("📊 Distribuciones en AP2:")
        create_distribution_plots(df, ap2_metrics, "Distribución AP2")
    
    # 4. COMPARACIONES ENTRE ASIGNACIONES
    print("\\n4️⃣ Comparaciones entre AP1 y AP2")
    print("="*60)
    
    # Boxplots comparativos
    available_base_metrics = [metric for metric in main_quality_metrics 
                             if f'{metric}_AP1' in df.columns and f'{metric}_AP2' in df.columns]
    
    if available_base_metrics:
        print("📊 Boxplots comparativos AP1 vs AP2:")
        create_comparison_boxplots(df, available_base_metrics)
    
    # 5. ANÁLISIS DE CORRELACIONES
    print("\\n5️⃣ Análisis de Correlaciones")
    print("="*60)
    
    # Matriz de correlación para métricas de AP1
    ap1_cols = [col for col in df.columns if col.endswith('_AP1') and df[col].dtype in ['int64', 'float64']]
    if len(ap1_cols) > 1:
        print("📊 Matriz de correlación - AP1:")
        corr_matrix_ap1 = df[ap1_cols].corr()
        
        plt.figure(figsize=(12, 8))
        sns.heatmap(corr_matrix_ap1, annot=True, cmap='RdBu_r', center=0, 
                   square=True, fmt='.2f', cbar_kws={'label': 'Correlación'})
        plt.title('Matriz de Correlación - Métricas AP1')
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.show()
    
    # Matriz de correlación para métricas de AP2
    ap2_cols = [col for col in df.columns if col.endswith('_AP2') and df[col].dtype in ['int64', 'float64']]
    if len(ap2_cols) > 1:
        print("📊 Matriz de correlación - AP2:")
        corr_matrix_ap2 = df[ap2_cols].corr()
        
        plt.figure(figsize=(12, 8))
        sns.heatmap(corr_matrix_ap2, annot=True, cmap='RdBu_r', center=0, 
                   square=True, fmt='.2f', cbar_kws={'label': 'Correlación'})
        plt.title('Matriz de Correlación - Métricas AP2')
        plt.xticks(rotation=45, ha='right')
        plt.yticks(rotation=0)
        plt.tight_layout()
        plt.show()
    
    print("\\n✅ Análisis Exploratorio de Datos completado")

else:
    print("❌ No se puede realizar EDA sin datos")

# 📊 Análisis Estadístico Avanzado de Calidad de Código en Proyectos Estudiantiles

## 🎯 Objetivo Principal

Desarrollar un análisis estadístico riguroso y comprehensivo que determine si existe una **mejora estadísticamente significativa** en la calidad del código entre **Programación Aplicada I (AP1)** y **Programación Aplicada II (AP2)**, identificando patrones, factores de influencia y recomendaciones pedagógicas.

## 📋 Datos Disponibles

- ✅ **60 estudiantes** con proyectos reales de programación
- ✅ **Métricas de SonarCloud**: bugs, vulnerabilidades, code smells, deuda técnica, complejidad, etc.
- ✅ **Issues detallados**: tipos, severidades, reglas violadas, ubicaciones específicas
- ✅ **Datos demográficos**: género, semestre, repositorios originales
- ✅ **Proyectos diversos**: sistemas de ventas, apps móviles, e-commerce, etc.

## 🔬 Metodología

1. **Análisis Exploratorio de Datos (EDA)**
2. **Pruebas de Hipótesis Múltiples**
3. **Análisis Multivariado y Machine Learning**
4. **Visualizaciones Avanzadas**
5. **Recomendaciones Pedagógicas**

---
*Análisis desarrollado para investigación en calidad de software educativa - Tesis Aplicada 2*